# PSyKE's demo for regression tasks

Some imports.

In [1]:
from psyke import Extractor
from psyke.regression import Grid, FixedStrategy
from sklearn.neighbors import KNeighborsRegressor
from psyke.utils.logic import pretty_theory
from sklearn.model_selection import train_test_split
import pandas as pd

Import an artificial dataset.

In [2]:
dataset = pd.read_csv("test/resources/datasets/arti.csv")

Split between train and test set in a reproducible way.

In [3]:
train, test = train_test_split(dataset, test_size = 0.3, random_state = 10)

We use as predictor a MLP and we train it.

In [4]:
predictor = KNeighborsRegressor(n_neighbors=1).fit(train.iloc[:, :-1].values, train.iloc[:, -1].values)

print((abs(predictor.predict(test.iloc[:, :-1].values).flatten() - test.iloc[:, -1].values)).mean())

0.009333333333333332


We create an extractor that uses the ITER algorithm and we extract prolog rules from our trained MLP.

In [5]:
it = Extractor.iter(predictor, min_update=1.0 / 15, n_points=4, max_iterations=600,
                    min_examples=100, threshold=0.15)
theory_from_iter = it.extract(train)
print('ITER extracted rules:\n\n' + pretty_theory(theory_from_iter))

ITER extracted rules:

z(X, Y, 0.0) :-
    X in [0.5095, 0.9994], Y in [0.5003, 0.9994].
z(X, Y, 0.3) :-
    X in [0.5095, 0.9994], Y in [0.0002, 0.5003].
z(X, Y, 0.4) :-
    X in [0.0021, 0.5095], Y in [0.4942, 0.9994].
z(X, Y, 0.7) :-
    X in [0.0021, 0.5095], Y in [0.0002, 0.4942].


In [6]:
gridEx = Extractor.gridex(predictor, Grid(1, FixedStrategy(2)), threshold=0.15)
theory_from_gridEx = gridEx.extract(train)
print('GridEx extracted rules:\n\n' + pretty_theory(theory_from_gridEx))

GridEx extracted rules:

z(X, Y, 0.6976) :-
    X in [0.0021, 0.5007], Y in [0.0002, 0.4998].
z(X, Y, 0.3984) :-
    X in [0.0021, 0.5007], Y in [0.4998, 0.9994].
z(X, Y, 0.2976) :-
    X in [0.5007, 0.9994], Y in [0.0002, 0.4998].
z(X, Y, 0.0) :-
    X in [0.5007, 0.9994], Y in [0.4998, 0.9994].


In [7]:
gridREx = Extractor.gridrex(predictor, Grid(1, FixedStrategy(2)), threshold=0.15)
theory_from_gridREx = gridREx.extract(train)
print('GridREx extracted rules:\n\n' + pretty_theory(theory_from_gridREx))

GridREx extracted rules:

z(X, Y, Z) :-
    X in [0.0021, 0.5007], Y in [0.0002, 0.4998], Z is 0.7052 + -0.004 * X + -0.0264 * Y.
z(X, Y, Z) :-
    X in [0.0021, 0.5007], Y in [0.4998, 0.9994], Z is 0.4164 + -0.022 * X + -0.017 * Y.
z(X, Y, Z) :-
    X in [0.5007, 0.9994], Y in [0.0002, 0.4998], Z is 0.2971 + 0.0103 * X + -0.0301 * Y.
z(X, Y, Z) :-
    X in [0.5007, 0.9994], Y in [0.4998, 0.9994], Z is 0.0.
